### 作者问题（2021年1月）
Macro for partially quoting (or partially evaluating) an expression

**先不考虑 可忽略，还看不太懂**

https://discourse.julialang.org/t/macro-for-partially-quoting-or-partially-evaluating-an-expression/53519

In [8]:
"""
e.g.

```julia
julia> eval(uneval(Expr(:my_call, :arg1, :arg2)))
:($(Expr(:my_call, :arg1, :arg2)))

julia> eval(eval(uneval(:(sqrt(9)))))
3.0
```
"""
function uneval(x::Expr)
    # the `Expr` below is assumed to be available in the scope and to be `Base.Expr`
    :(Expr($(uneval(x.head)), $(map(uneval, x.args)...)))
end

# tangential TODO: determine which one
uneval(x) = Meta.quot(x)
# uneval(x) = Meta.QuoteNode(x)

"""
e.g.
julia> dump(let x = 9
       @xquote sqrt(x)
       end)
Expr
    head: Symbol call
    args: Array{Any}((2,))
        1: sqrt (function of type typeof(sqrt))
        2: Int64 9
"""
macro xquote(ex)
    uneval(ex) # TODO escape `::Symbol` in `args`.
end

@xquote

In [9]:
dump(let x = 9
    @xquote sqrt(x)
    end)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol sqrt
    2: Symbol x


but I would like the output to be what I wrote in the docstring for `@xquote`. For this particular expression, the following definition produces the desired output:

In [10]:
macro xquote_sqrt9(ex)
    uneval(Expr(:call, sqrt, 9))
end

@xquote_sqrt9 (macro with 1 method)

as does explicitly quoting:<br>
就像明确引用一样

In [12]:
dump(let x = 9
    :($(sqrt)($x))
    end)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: sqrt (function of type typeof(sqrt))
    2: Int64 9


I’ve played around with `f(x) = esc(x)`, and `f(x) = Expr(:$, x)` in
```julia
escape_all_symbols(ex) = 
MacroTools.postwalk(
    x -> x isa Symbol ? f(x) : x, ex
    )
```
but I’m a bit lost about how to move forward.<br><br>

I think this is tangential to my question, but I won’t be evaluating every single `Symbol`. Specifically, I would not evaluate `Symbol` s that correspond to function arguments. For example, currently:<br>

我想这和我的问题有点关系，但我不会计算每一个 `Symbol` 。具体来说，我不会计算 `Symbol` s对应于函数参数。例如，目前：

In [13]:
dump(let x = 9
    @xquote y -> x + y
    end)

Expr
  head: Symbol ->
  args: Array{Any}((2,))
    1: Symbol y
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 2
          file: Symbol e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Symbol x
            3: Symbol y


And in my desired output, `Symbol +` and `Symbol x` would be evaluated, but `Symbol y` would not be. I think I’ve got that aspect covered, however, and it would be enough for my purposes to see a definition of `@xquote` where, for example every `Symbol` except for `:y` is evaluated.<br>

在我想要的输出中， `Symbol +` 和 `Symbol x` 会被求值，但是 `Symbol y` 不会被求值。然而，我认为我已经涵盖了这方面，并且对于我的目的来说，看到 `@xquote` 的定义就足够了，例如，除了 `:y` 之外的每个符号都被评估。

### 作者自己回复

I made a mistake in the original post. That special case macro should not have been<br>
我在原来的帖子中犯了一个错误。这个特殊情况下的宏不应该存在
```julia
macro xquote_sqrt9(ex)
    uneval(Expr(:call, sqrt, 9))
end
```
but instead 而是
```julia
macro xquote_sqrt_x(ex)
    uneval(Expr(:call, esc(:sqrt), 9))
end
```
I believe I have answered my own question:<br>
我想我已经回答了我自己的问题：

In [1]:
using MacroTools

In [2]:
"""
e.g.

```julia
julia> eval(uneval(Expr(:my_call, :arg1, :arg2)))
:($(Expr(:my_call, :arg1, :arg2)))

julia> eval(eval(uneval(:(sqrt(9)))))
3.0
```

Note the special case for `:(esc(x))`.
"""
function uneval(x::Expr)
    x.head === :escape && return x
    # the `Expr` below is assumed to be available in the scope and to be `Base.Expr`
    :(Expr($(uneval(x.head)), $(map(uneval, x.args)...)))
end

uneval

In [3]:
# tangential TODO: determine which one
uneval(x) = Meta.quot(x)
# uneval(x) = Meta.QuoteNode(x)

uneval (generic function with 2 methods)

In [4]:
escape_all_symbols(ex) = 

MacroTools.postwalk(
    x -> x isa Symbol ? esc(x) : x, ex
    )

escape_all_symbols (generic function with 1 method)

In [5]:
"""
e.g.
julia> dump(let x = 9
       @xquote sqrt(x)
       end)
Expr
    head: Symbol call
    args: Array{Any}((2,))
        1: sqrt (function of type typeof(sqrt))
        2: Int64 9
"""
macro xquote(ex)
    uneval(escape_all_symbols(ex))
end

@xquote

In [6]:
dump(let x = 9
     @xquote sqrt(x)
     end)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: sqrt (function of type typeof(sqrt))
    2: Int64 9


In [7]:
dump(let x = 9, sqrt=sin
    @xquote sqrt(x)
    end)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: sin (function of type typeof(sin))
    2: Int64 9
